# PROJET MACH_BDA_DATAVIZ

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from datasets import load_dataset
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
dataset = load_dataset("d0r1h/customer_churn")

df = pd.DataFrame(dataset['train'])

df_reset = df.copy() # copie pour ne pas recharger tout le dataset

In [ ]:
def reset_dataframe():
    return df_reset

In [ ]:
df.iloc[0]

## Traitement des Données

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
def clean_dataframe(df):
    df.dropna(inplace=True)
    df.drop(columns=['security_no', 'referral_id'],inplace=True)
    
    df['medium_of_operation'] = df['medium_of_operation'].replace('?', 'Unknown')
    
    df = df[df['days_since_last_login'] > 0]
    df = df[df['avg_time_spent'] > 0]
    df = df[df['points_in_wallet'] > 0]
    df = df[df['avg_frequency_login_days'] != "Error"]
    
    df['avg_frequency_login_days'] = pd.to_numeric(df['avg_frequency_login_days'])
    
    df['joining_date'] = pd.to_datetime(df['joining_date'], format='%d-%m-%Y')
    df['last_visit_time'] = pd.to_datetime(df['last_visit_time'], format='%H:%M:%S').dt.time

    df['joining_year'] = df['joining_date'].dt.year
    df['joining_month'] = df['joining_date'].dt.month
    df['joining_day'] = df['joining_date'].dt.day

    df['last_visit_hour'] = df['last_visit_time'].apply(lambda x: x.hour)
    df['last_visit_minute'] = df['last_visit_time'].apply(lambda x: x.minute)

    df.drop(columns=['joining_date', 'last_visit_time'], inplace=True)
    
    return df

Standardisation des données numériques (mettre toutes les valeurs entre 0 et 1, pas sûr s'il y a vraiment besoin):

In [ ]:
def normalize_numeric_columns(df):
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = MinMaxScaler().fit_transform(df[numeric_columns])
    return df

Encodage des données non numériques:

In [ ]:
def encode_columns(df):
    non_numeric_columns = df.select_dtypes(include=['object']).columns.tolist()
    
    dict_encode = { value: index for index, value in enumerate(non_numeric_columns) }
    
    encoded_df = df.copy()
    
    for col in dict_encode:
        l_occurences = df[col].unique()
        dict_encode[col] = { value: index for index, value in enumerate(l_occurences) }
        encoded_df[col] = encoded_df[col].apply(lambda x : dict_encode[col][x])
    
    return encoded_df

In [ ]:
# df = reset_dataframe()

In [ ]:
df = clean_dataframe(df)

Je n'ai pas encore reussi a encoder les colonnes non numeriques

## Analyse et visualisation exploratoire des Données

Matrice de corrélation pour voir si certaines features sont redondantes

In [ ]:
correlation_matrix = encode_columns(df).corr()
plt.figure(figsize=(25, 25))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice de corrélation')
plt.show()

## Evaluation de plusieurs modèles avec les bonnes métriques

In [ ]:
df = encode_columns(df)
df = normalize_numeric_columns(df)

In [ ]:
X = df.drop(columns=['churn_risk_score'])
y = df['churn_risk_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(kernel='linear'),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'AdaBoost': AdaBoostClassifier(),
}

results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    results[model_name] = accuracy

for model_name, accuracy in results.items():
    print(f"{model_name} Accuracy: {accuracy:.2f}")


Optimisation des meilleurs modèles (GDBoosting et RandomForest) pour améliorer leur accuracy:

In [ ]:
model = GradientBoostingClassifier()

param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

random_search_gdb = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42, scoring='accuracy')

random_search_gdb.fit(X_train, y_train)

In [ ]:
print("Best parameters found: ", random_search_gdb.best_params_)

print("Best cross-validation score: {:.2f}".format(random_search_gdb.best_score_))

best_model_gdb = random_search_gdb.best_estimator_
y_pred_gdb = best_model_gdb.predict(X_test)
accuracy_gdb = accuracy_score(y_test, y_pred_gdb)

print("Test set accuracy: {:.2f}".format(accuracy_gdb))

Best parameters found: {'subsample': 1.0, 'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 5, 'learning_rate': 0.1}
Best cross-validation score: 0.94
Test set accuracy: 0.95

In [ ]:
model = RandomForestClassifier()

param_distributions = {
    'n_estimators': np.arange(100, 501, 50),
    'max_depth': np.arange(3, 21, 2),
    'min_samples_split': np.arange(2, 21, 2),
    'min_samples_leaf': np.arange(1, 11, 1),
    'bootstrap': [True, False]
}

random_search_rf = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=100, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
random_search_rf.fit(X_train, y_train)

In [ ]:
print("Best parameters found: ", random_search_rf.best_params_)

print("Best cross-validation score: {:.2f}".format(random_search_rf.best_score_))

best_model_rf = random_search_rf.best_estimator_
y_pred_rf = best_model_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("Test set accuracy: {:.2f}".format(accuracy_rf))

Best parameters found:  {'n_estimators': 250, 'min_samples_split': 16, 'min_samples_leaf': 3, 'max_depth': 19, 'bootstrap': False}
Best cross-validation score: 0.94
Test set accuracy: 0.94

In [ ]:
def features_importance(model,coeff_method=None):
    if coeff_method == 'arbres':
        feature_importances = model.feature_importances_
    else:
        feature_importances = model.coef_[0]
    
    feature_importance_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': feature_importances
    }).sort_values(by='Importance', ascending=False)

    return feature_importance_df

In [ ]:
feature_importance_df = features_importance(models['Random Forest'], 'arbres')

In [ ]:
feature_importance_df

## Visualiser et interpreter les résultats

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()